### formatted_json_recipeの内容をWv2向けのトークンにして保存する

In [75]:
from neo4j import GraphDatabase
import pandas as pd
from openai import OpenAI
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import networkx as nx
import json
from collections import Counter
import os
from itertools import combinations
from dowhy import CausalModel
import re
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import warnings
from typing import Optional, List, Tuple, Dict
import numpy as np
from dataclasses import dataclass
from sklearn.preprocessing import MinMaxScaler

load_dotenv()

# NumPy関連のRuntimeWarningを無効化
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [76]:
# DB接続処理
# Neo4jに接続
uri = "bolt://localhost:7687"
user = "neo4j"
password = "abcd7890"

# ドライバを作成
driver = GraphDatabase.driver(uri, auth=(user, password))

In [77]:
# トークン化して保存する
def tokenize_save(key_name, sentences):
    # NumPyの配列に変換
    tokens = []
    for sentence in sentences:
        if sentence is None:
            continue
        
        if isinstance(sentence, list):
            sentence = " ".join(sentence)
            
        # 小文字化し、特殊文字や句読点を削除してトークン化
        words = re.findall(r'\b[a-zA-Z0-9_]+\b', sentence.lower())
        tokens.append(words)
        
    np_sentences = np.array(tokens, dtype=object)

    # NumPyの配列をdumpして保存
    np.save(f"../data/word2_vec/{key_name}.npy", np_sentences)
    with open(f"../data/word2_vec/{key_name}.txt", 'w') as f:
        json.dump(tokens, f)

In [78]:
# テキストフォーマットの指定
def format_text(text):
    if text is None:
        return ""
    text = text.replace('(', '_')
    text = text.replace(')', '_')
    text = text.replace("/", '_')
    text = text.replace(";", '_')
    text = text.replace(":", '_')
    text = text.replace("&", '_')
    text = text.replace("[", '')
    text = text.replace("]", '')    
    text = text.replace('"', '')
    text = text.replace('<', '')
    text = text.replace('>', '')
    text = text.replace(', ', ',')
    text = text.replace('.', '')
    text = text.replace(',', '_')
    text = text.replace('-', ' ')
    text = text.replace(' ', '_')
    text = text.replace('\n', '')
    text = text.replace('%', '')
    text = text.replace("'", '')
    text = text.replace("/t", '')
    text = text.replace("\\", '')
    text = text.replace("é", '')
    text = text.replace("ç", '')
    text = text.replace("+", '')
    text = text.replace("`", '')
    text = text.replace("`", '')
    text = text.replace("`", '')
    text = text.replace("`", '')
    text = text.replace("ã", '')
    text = text.replace("â", '')
    text = text.replace("ƒ", '')
    text = text.replace("€", '')
    text = text.replace('”', '')
    text = text.replace('`', '')
    text = text.replace('!', '')
    text = text.replace('|', '')
    text = text.replace("\n", "")
    text = text.replace("\r", "")
    text = re.sub(r'\s+', '', text)
    text = text.lower()
    return text

In [79]:
def find_food(tx, search_term):
    find_food_query = """
    CALL db.index.fulltext.queryNodes("food_origin_index_text_search", $search_term)
    YIELD node, score as full_text_score
    ORDER BY full_text_score DESC, size(node.name)
    LIMIT 1
    RETURN node.id as node_id, node.name as node_name, node.flavor_vector as flavor_vector, node.word_vector as word_vector, full_text_score    
    """
    result = tx.run(find_food_query, search_term=search_term).data()
    if result:
        return result[0]
        
    return None

In [80]:
new_json_data = {}
with open("../data/formatted_json_recipe.json", "r") as file:
    new_json_data = json.load(file)
print(len(new_json_data))

1325


In [81]:
new_rows = []
counter = 0

for key, row in new_json_data.items():
    #counter += 1
    #if counter > 10:
    #    break
    
    if not row["ingredients"]:  # 空リストのチェック
        continue
    
    with driver.session() as session:
        for ingri in row["ingredients"]:
            recipe = []
            result = find_food(session, search_term=ingri)
            if result:
                # タプル形式でデータを追加
                recipe.append(result["node_id"])
            new_rows.append(recipe)

In [82]:
tokenize_save("formatted_recipe_ingridients", new_rows)